In [1]:
import serial
import time
import struct

# List Important Registers
Sourced from [datasheet](https://statics.cirrus.com/pubs/proDatasheet/CS5490_F3.pdf)

In [2]:
CMD_READ  = 0x00
CMD_WRITE = 0x40
CMD_PAGE  = 0x80
CMD_INST  = 0xC0

# HARDWARE Page 0
HARDWARE_PAGE = 0 << 8

# Note, page + address are merged into a single 16 bit value
# List Addresses
CONFIG0 = 0x00 + HARDWARE_PAGE
CONFIG1 = 0x01 + HARDWARE_PAGE
MASK = 0x03 + HARDWARE_PAGE
PC = 0x05 + HARDWARE_PAGE
SERIALCTRL = 0x07 + HARDWARE_PAGE
PULSEWIDTH = 0x08 + HARDWARE_PAGE
PULSECTRL = 0x09 + HARDWARE_PAGE
STATUS0 = 0x17 + HARDWARE_PAGE
STATUS1 = 0x18 + HARDWARE_PAGE
STATUS2 = 0x19 + HARDWARE_PAGE
REGLOCK = 0x22 + HARDWARE_PAGE
VPEAK = 0x24 + HARDWARE_PAGE
IPEAK = 0x25 + HARDWARE_PAGE
PSDC = 0x30 + HARDWARE_PAGE
ZXNUM = 0x37 + HARDWARE_PAGE

# SOFTWARE Page 16
SOFTWARE_P16 = 16 << 8

# List Addresses
CONFIG2 = 0x00 + SOFTWARE_P16
REGCHK = 0x01 + SOFTWARE_P16
V_INST = 0x02 + SOFTWARE_P16
I_INST = 0x03 + SOFTWARE_P16
P_INST = 0x04 + SOFTWARE_P16
P_AVG = 0x05 + SOFTWARE_P16
I_RMS = 0x06 + SOFTWARE_P16
V_RMS = 0x07 + SOFTWARE_P16
Q_AVG = 0x0E + SOFTWARE_P16
Q_INST = 0x0F + SOFTWARE_P16
S_INST = 0x14 + SOFTWARE_P16
PF = 0x15 + SOFTWARE_P16
TEMP = 0x1B + SOFTWARE_P16  # S Q7.16
P_SUM = 0x1D + SOFTWARE_P16
S_SUM = 0x1E + SOFTWARE_P16
Q_SUM = 0x1F + SOFTWARE_P16
I_DCOFF = 0x20 + SOFTWARE_P16
I_GAIN = 0x21 + SOFTWARE_P16  # U Q2.22
V_DCOFF = 0x22 + SOFTWARE_P16
V_GAIN = 0x23 + SOFTWARE_P16  # U Q2.22
P_OFF = 0x24 + SOFTWARE_P16
I_ACOFF = 0x25 + SOFTWARE_P16
EPSILON = 0x31 + SOFTWARE_P16
SAMPLECOUNT = 0x33 + SOFTWARE_P16
T_GAIN = 0x36 + SOFTWARE_P16  # U Q8.16
T_OFF = 0x37 + SOFTWARE_P16
T_SETTLE = 0x39 + SOFTWARE_P16
LOAD_MIN = 0x40 + SOFTWARE_P16
SYS_GAIN = 0x3C + SOFTWARE_P16
SYS_TIME = 0x3D + SOFTWARE_P16

# SOFTWARE Page 17
SOFTWARE_P17 = 17 << 8

# List Addresses
VSAG_DUR = 0x00 + SOFTWARE_P17
VSAG_LEVEL = 0x01 + SOFTWARE_P17
IOVER_DUR = 0x04 + SOFTWARE_P17
IOVER_LEVEL = 0x05 + SOFTWARE_P17

# SOFTWARE Page 18
SOFTWARE_P18 = 18 << 8

# List Addresses
IZX_LEVEL = 0x18 + SOFTWARE_P18
PULSERATE = 0x1C + SOFTWARE_P18
INT_GAIN = 0x2B + SOFTWARE_P18
VSWELL_DUR = 0x2E + SOFTWARE_P18
VSWELL_LEVEL = 0x2F + SOFTWARE_P18
VZX_LEVEL = 0x3A + SOFTWARE_P18
CYCLECOUNT = 0x3E + SOFTWARE_P18
SCALE = 0x3F + SOFTWARE_P18


# Start of register metadata, will help with register dumps, value conversions, sanity check
reginfo = {  # (string:name, int:q, bool:signed, int:default)
    # SW page 16
    CONFIG2: ('config2', 0, False, 0x100200),
    REGCHK: ('regchk', 0, False, 0),
    V_INST: ('v_inst', 23, True, 0),
    I_INST: ('i_inst', 23, True, 0),
    P_INST: ('p_inst', 23, True, 0),
    P_AVG: ('p_avg', 23, True, 0),
    I_RMS: ('i_rms', 24, False, 0),
    V_RMS: ('v_rms', 24, False, 0),
    Q_AVG: ('q_avg', 23, True, 0),

    TEMP: ('temp', 16, True, 0)
}


# Instructions (use with CMD_INST)

# Controls
SOFT_RESET = 0x1
STANDBY = 0x2
WAKEUP = 0x3
SINGLE_CONV =  0x14
CONT_CONV = 0x15
HALT_CONV = 0x18

# Calibration
I_AC_CAL = 0x31
V_AC_CAL = 0x32
IV_AC_CAL = 0x36
I_DC_CAL = 0x21
V_DC_CAL = 0x22
IV_DC_CAL = 0x26
I_GAIN_CAL = 0x39
V_GAIN_CAL = 0x3A

IV_GAIN_CAL = 0x3E

In [3]:
 # Conversion functions between float and fixed point

def q2f(v, q, signed=True):
    '''Signed fixed point to float conversion
    val = 24 bit integer value
    q = number of fraction bits 0..23'''
    sign = v & 0x800000
    if signed:
        assert(q <= 23)
        v = v & 0x7FFFFF
        v = v - sign
    else:
        assert(q <= 24)
    return v / (1 << q)


def f2q(val, q, signed=True):
    '''Float to fixed point. Works for signed and unsigned nubmers'''
    return int(0xFFFFFF & int(val * (1 << q)))

# Quick eyeball test
tv = (1, 0x10000, 0x7FFFFF, 0x800000, 0x810000, 0xFFFFFF)
for v in tv:
    s = q2f(v, 23, True)
    u = q2f(v, 24, False)
    print(v, s, u, f2q(s, 23, True), f2q(u, 24, False))

1 1.1920928955078125e-07 5.960464477539063e-08 1 1
65536 0.0078125 0.00390625 65536 65536
8388607 0.9999998807907104 0.4999999403953552 8388607 8388607
8388608 -1.0 0.5 8388608 8388608
8454144 -0.9921875 0.50390625 8454144 8454144
16777215 -1.1920928955078125e-07 0.9999999403953552 16777215 16777215


In [4]:
class cs5490(object):
    def __init__(self, serial, debug=False):
        self.current_page = None
        self.serial = serial
        self.debug = debug
        
    def read_register(self, register):
        page = (register & 0x3F00) >> 8
        addr = register & 0x3F
        if self.current_page != page:
            self.serial.write(bytearray([CMD_PAGE | page]))
            self.current_page = page
        self.serial.write(bytearray([addr]))
        time.sleep(0.1)
        buffer = bytearray(4)
        rb = self.serial.read(3)
        buffer[0:3] = rb
        val = struct.unpack('<L',buffer)[0]
        if self.debug:
            print('Read %02d:%02d = 0x%06x %d' % (page, addr, val, val))
        return val

    def write_register(self, register, val):
        page = (register & 0x3F00) >> 8
        addr = register & 0x3F
        if self.current_page != page:
            self.serial.write(bytearray([CMD_PAGE | page]))
            self.current_page = page
        self.serial.write(bytearray([CMD_WRITE | addr]))
        buffer = struct.pack('<L', val)
        self.serial.write(buffer[0:3])    
        if self.debug:
            print('Write %02d:%02d = 0x%06x %d' % (page, addr, val, val))
        
    def instruction(self, instr):
        self.serial.write(bytearray([CMD_INST | instr]))
        if self.debug:
            print('Instruction %x' % instr)

In [8]:
portname = '/dev/serial/by-id/usb-FTDI_FT232R_USB_UART_A6007wZa-if00-port0'
sp = serial.Serial(port=portname, baudrate=600)
time.sleep(1)
chip = cs5490(sp, debug=True)

# Perform Soft RESET

In [9]:
chip.instruction(SOFT_RESET)

Instruction 1


# Read Hardware register Config0

In [10]:
chip.read_register(CONFIG0)

Read 00:00 = 0xc02000 12591104


12591104

In [11]:
hw0_valid_addrs = (CONFIG0, CONFIG1, MASK, PC, SERIALCTRL,
PULSEWIDTH, PULSECTRL, STATUS0, STATUS1, STATUS2,
VPEAK, IPEAK, PSDC, ZXNUM)

chip.debug = True
for r in hw0_valid_addrs:
    rv = chip.read_register(r)
chip.debug =  False


Read 00:00 = 0xc02000 12591104
Read 00:01 = 0x00eeee 61166
Read 00:03 = 0x000000 0
Read 00:05 = 0x000000 0
Read 00:07 = 0x02004d 131149
Read 00:08 = 0x000001 1
Read 00:09 = 0x000000 0
Read 00:23 = 0x800000 8388608
Read 00:24 = 0x801800 8394752
Read 00:25 = 0x000000 0
Read 00:36 = 0x000000 0
Read 00:37 = 0x000000 0
Read 00:48 = 0x000000 0
Read 00:55 = 0x000064 100


In [12]:
for addr, info in reginfo.items():
    v = chip.read_register(addr)
    print(addr, info[0], v, info[3], q2f(v, info[1], info[2]))

4096 config2 1049088 1049088 1049088.0
4097 regchk 0 0 0.0
4098 v_inst 0 0 0.0
4099 i_inst 0 0 0.0
4100 p_inst 0 0 0.0
4101 p_avg 0 0 0.0
4102 i_rms 0 0 0.0
4103 v_rms 0 0 0.0
4123 temp 0 0 0.0
4110 q_avg 0 0 0.0


# Read Software register Voltage Gain, Current Gain, Temperature Gain

In [13]:
v = chip.read_register(V_GAIN)
print(q2f(v, 22, False))

1.0


In [14]:
gain = 1.0
gain_q = f2q(gain, 22, False)
chip.write_register(V_GAIN, gain_q)

In [15]:
v = chip.read_register(V_GAIN)
print(q2f(v, 22, False))

1.0


In [16]:
i = chip.read_register(I_GAIN)
print(q2f(i, 22, False))

1.0


In [17]:
t = chip.read_register(T_GAIN)
print(q2f(t, 16, False))

6.715179443359375


# Read Software register Voltage Sag, Over Current

In [18]:
_ = chip.read_register(VSAG_LEVEL)
_ = chip.read_register(IOVER_LEVEL)

# Read Instantenous voltage and current

In [19]:
chip.instruction(CONT_CONV)

In [20]:
chip.debug = False
for _ in range(10):
    v = chip.read_register(V_RMS)
    i = chip.read_register(I_RMS)
    p = chip.read_register(P_AVG)

    print(q2f(v, 24, False), q2f(i, 24, False), q2f(p, 23))
    time.sleep(1)

0.2146877646446228 0.3998343348503113 0.08568859100341797
0.2147316336631775 0.39984428882598877 0.08571052551269531
0.21479225158691406 0.3999149799346924 0.08575296401977539
0.2148202657699585 0.40001779794692993 0.0858309268951416
0.21478945016860962 0.3998774290084839 0.08575713634490967
0.21478533744812012 0.3998327851295471 0.08575189113616943
0.21497714519500732 0.39999741315841675 0.08594214916229248
0.2148294448852539 0.39984625577926636 0.08578479290008545
0.21470844745635986 0.3996877670288086 0.08570265769958496
0.2147221565246582 0.3997555375099182 0.08572304248809814


In [21]:
reg = TEMP
name, q, sign, default = reginfo[TEMP]
t = chip.read_register(TEMP)
print(name, q2f(t, q, sign))

temp 26.08984375
